In [3]:
import pandas as pd
import os
import time
import math
from numbers import Number
import argparse

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
from PIL import Image
from torch.utils.data import DataLoader
from torchvision import datasets
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import brewer2mpl
from tqdm import tqdm



from importlib import reload




## Get training data

In [4]:
df = pd.read_csv('data/XOM_norm_data.csv', sep = ',', header = 0)


# date.  input features   x.     delta.   label (0,1,2)



X_train = []
y_train = []
for i in range(60, len(training_set_scaled)):
    X_train.append(training_set_scaled[i-60: i, 0])
    y_train.append(training_set_scaled[i, 0])
    X_train, y_train = np.array(X_train), np.array(y_train)

NameError: name 'training_set_scaled' is not defined

In [9]:
df['Date'].min()

'01/02/2021'

In [ ]:


class Object(object):
    pass

args = Object()
args.dataset = 'mnist'
args.dist = 'normal'
args.num_epochs = 10
args.batch_size = 32
args.learning_rate = 1e-3
args.latent_dim = 10
args.beta = 10
args.tcvae = True
args.exclude_mutinfo = False
args.beta_anneal = False
args.lambda_anneal = False
args.mss = False
args.conv = True
args.gpu = 0
args.visdom = False
args.save = 'mnist_10_10'
args.log_freq = 200






In [ ]:
import wandb
wandb.login()

In [ ]:
wandb.init(project="stock_pred")

In [ ]:
# for loading and batching datasets
def setup_data_loaders(args, use_cuda=False):
    from torchvision.transforms import ToTensor, Lambda
    from torchvision import datasets


    if args.dataset == 'shapes':
        train_set = dset.Shapes()
    elif args.dataset == 'faces':
        train_set = dset.Faces()
    elif args.dataset == 'mnist':
        train_set = datasets.MNIST(
        root="data",
        train=True,
        download=True,
        transform=ToTensor())

    else:
        raise ValueError('Unknown dataset ' + str(args.dataset))

    kwargs = {'num_workers': 0, 'pin_memory': use_cuda}


    train_loader = DataLoader(dataset=train_set,
        batch_size=args.batch_size, shuffle=True, **kwargs)
    return train_loader

class SiameseDataLoader(Dataset):
    """
    Train: For each sample creates randomly a positive or a negative pair
    Test: Creates fixed pairs for testing
    """

    def __init__(self, mnist_dataset):
        self.mnist_dataset = mnist_dataset

        self.train = self.mnist_dataset.train
        self.transform = self.mnist_dataset.transform

        if self.train:
            self.train_labels = self.mnist_dataset.train_labels
            self.train_data = self.mnist_dataset.train_data
            self.labels_set = set(self.train_labels.numpy())
            self.label_to_indices = {label: np.where(self.train_labels.numpy() == label)[0]
                                     for label in self.labels_set}
       
    def __getitem__(self, index):
        if self.train:
            target = np.random.randint(0, 2)
            img1, label1 = self.train_data[index], self.train_labels[index].item()
            if target == 1:
                siamese_index = index
                while siamese_index == index:
                    siamese_index = np.random.choice(self.label_to_indices[label1])
            else:
                siamese_label = np.random.choice(list(self.labels_set - set([label1])))
                siamese_index = np.random.choice(self.label_to_indices[siamese_label])
            img2 = self.train_data[siamese_index]
        
        img1 = Image.fromarray(img1.numpy(), mode='L')
        img2 = Image.fromarray(img2.numpy(), mode='L')
        if self.transform is not None:
            img1 = self.transform(img1)
            img2 = self.transform(img2)
        return (img1, img2), target

    def __len__(self):
        return len(self.mnist_dataset)

    

In [ ]:
class EmbeddingNet(nn.Module):
    def __init__(self):
        super(EmbeddingNet, self).__init__()
        self.convnet = nn.Sequential(nn.Conv2d(1, 32, 5), nn.PReLU(),
                                     nn.MaxPool2d(2, stride=2),
                                     nn.Conv2d(32, 64, 5), nn.PReLU(),
                                     nn.MaxPool2d(2, stride=2))

        self.fc = nn.Sequential(nn.Linear(64 * 4 * 4, 256),
                                nn.PReLU(),
                                nn.Linear(256, 256),
                                nn.PReLU(),
                                nn.Linear(256, 2)
                                )

    def forward(self, x):
        output = self.convnet(x)
        output = output.view(output.size()[0], -1)
        output = self.fc(output)
        return output

    def get_embedding(self, x):
        return self.forward(x)

class SiameseNet(nn.Module):
    def __init__(self, embedding_net):
        super(SiameseNet, self).__init__()
        self.embedding_net = embedding_net

    def forward(self, x1, x2):
        output1 = self.embedding_net(x1)
        output2 = self.embedding_net(x2)
        return output1, output2

    def get_embedding(self, x):
        return self.embedding_net(x)

class ClassificationNet(nn.Module):
    def __init__(self, embedding_net, n_classes):
        super(ClassificationNet, self).__init__()
        self.embedding_net = embedding_net
        self.n_classes = n_classes
        self.nonlinear = nn.PReLU()
        self.fc1 = nn.Linear(2, n_classes)

    def forward(self, x):
        output = self.embedding_net(x)
        output = self.nonlinear(output)
        scores = F.log_softmax(self.fc1(output), dim=-1)
        return scores

    def get_embedding(self, x):
        return self.nonlinear(self.embedding_net(x))

In [ ]:
# torch.cuda.set_device(args.gpu)
use_cuda = False
# data loader
train_loader = setup_data_loaders(args, use_cuda=False)



# setup the VAE
# if args.dist == 'normal':
#     prior_dist = dist.Normal()
#     q_dist = dist.Normal()
# elif args.dist == 'laplace':
#     prior_dist = dist.Laplace()
#     q_dist = dist.Laplace()
# elif args.dist == 'flow':
#     prior_dist = FactorialNormalizingFlow(dim=args.latent_dim, nsteps=32)
#     q_dist = dist.Normal()


model = SiameseNet()


# vae = VAE(z_dim=args.latent_dim, use_cuda=use_cuda, prior_dist=prior_dist, q_dist=q_dist, image_dim = 28,
#     include_mutinfo=not args.exclude_mutinfo, tcvae=args.tcvae, conv=args.conv, mss=args.mss)

# setup the optimizer
optimizer = optim.Adam(vae.parameters(), lr=args.learning_rate)
criterion = torch.nn.CrossEntropyLoss(loss_weights)


In [ ]:
# log info
train_log_string = '%s :: Epoch %i :: Iter %i / %i :: train loss: %0.4f'
valid_log_string = '%s :: Epoch %i :: valid loss: %0.4f\n'

# Restore saved model (if one exists).
ckpt_path = os.path.join(args.ckpt_dir, args.exp_name+'.pt')

if os.path.exists(ckpt_path):
    print('Loading checkpoint: %s' % ckpt_path)
    ckpt = torch.load(ckpt_path)
    epoch = ckpt['epoch']
    model.load_state_dict(ckpt['siamese'])
    optimizer.load_state_dict(ckpt['optimizer'])
else:
    epoch = 1
    print('Fresh start!\n')


if torch.cuda.is_available():
    criterion = criterion.cuda()
    model = model.cuda()


    
# save every epoch for visualization
train_loss_record = []
valid_loss_record = []
best_record = 10.0

# training
print('Experiment: {}\n'.format(config['experiment_name']))

while epoch < args.num_epochs:

    print('Start Epoch {} Training...'.format(epoch))

    # loss
    train_loss = []
    train_loss_sum = []
    # dataloader
    train_dataloader = DataLoader(dataset=trainDS, shuffle=True, num_workers=1, batch_size=32)

    for idx, data in enumerate(train_dataloader, 0):
        print('idx:', idx)
        # get data
        s1, s2, label = data

        # clear gradients
        optimizer.zero_grad()

        # input
        output = model(s1, s2)
        output = output.squeeze(0)

        # label cuda
        label = Variable(label)
        if torch.cuda.is_available():
            label = label.cuda()

        # loss backward
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        train_loss.append(loss.data.cpu())
        train_loss_sum.append(loss.data.cpu())
        
        wandb.log({'train_loss': loss.data.cpu(), 'iteration': idx, 'epoch': epoch})

            
            
        # Every once and a while check on the loss
        if ((idx + 1) % 5000) == 0:
            print(train_log_string % (datetime.now(), epoch, idx + 1, len(train), np.mean(train_loss)))
            train_loss = []

    # Record at every epoch
    print('Train Loss at epoch {}: {}\n'.format(epoch, np.mean(train_loss_sum)))
    train_loss_record.append(np.mean(train_loss_sum))


In [ ]:

train_elbo = []

# training loop
dataset_size = len(train_loader.dataset)
num_iterations = len(train_loader) * args.num_epochs
iteration = 0
# initialize loss accumulator
elbo_running_mean = utils.RunningAverageMeter()
while iteration < num_iterations:
    for i, x in enumerate(train_loader):
        iteration += 1
        print(iteration)
        batch_time = time.time()
        vae.train()
        utils.anneal_kl(args, vae, iteration)
        optimizer.zero_grad()
        # transfer to GPU
        x = x[0]
        # print(type(x))
        # print(x)
        # print(len(x), len(x[0]), len(x[0][0]), len(x[0][0][0]))
        if use_cuda:
            x = x.cuda()
        # wrap the mini-batch in a PyTorch Variable
        x = Variable(x)
        # do ELBO gradient and accumulate loss
        obj, elbo = vae.elbo(x, dataset_size, 28)
#         print('elbo', elbo)
        if utils.isnan(obj).any():
            raise ValueError('NaN spotted in objective.')
        obj.mean().mul(-1).backward()
        elbo_running_mean.update(torch.mean(elbo.mean()))
        optimizer.step()
        
        wandb.log({'elbo_rm': elbo_running_mean.avg, 'iteration': iteration, 'obj': obj.mean().item()})


        # report training diagnostics
        if iteration % args.log_freq == 0:
            train_elbo.append(elbo_running_mean.avg)
            print('[iteration %03d] time: %.2f \tbeta %.2f \tlambda %.2f training ELBO: %.4f (%.4f)' % (
                iteration, time.time() - batch_time, vae.beta, vae.lamb,
                elbo_running_mean.val, elbo_running_mean.avg))

            vae.eval()

            # plot training and test ELBOs
#             if args.visdom:
            utils_plot.display_samples(vae, x, vis, 28)
            utils_plot.plot_elbo(train_elbo, vis)

            utils.save_checkpoint({
                'state_dict': vae.state_dict(),
                'args': args}, args.save, 0)
            eval('plot_vs_gt_' + args.dataset)(vae, train_loader.dataset,
                                               os.path.join(args.save, 'gt_vs_latent_{:05d}.png'.format(iteration)),
                                               use_cuda=False)

